In [1]:
import sys
import scipy
import numpy
import matplotlib
import pandas
import sklearn
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Datos con todos los incidentes

In [59]:
url = "/Users/vrandkode/Workspace/M2/CienciaDeDatosII/datasets/incidents.all.ml.csv"
names = ['category', 'type', 'dayofweek', 'date', 'time','day','year','month','hour','district', 'resolution', 'latitude', 'longitude']
dataset = pandas.read_csv(url, names=names, sep=';')


# Modelos de clasificación
* Random forests
* Neural networks
* Klustering

## Random forests


## Klustering

## Neural networks

Con ayuda de una red neuronal en Python intentaremos averigüar si dada un distrito y dia de la semana hay posibilidades de producirse un tipo de delito o no. Usaremos el conjunto de datos de incidentes del Repositorio de aprendizaje automático de UCI. Intentaremos construir un modelo que pueda clasificar en función de las situaciones: distrito, dia de la semana  e intentar predecir el tipo de delito que pueda producirse por la zona utilizando redes neuronales.


* Descarte de las columnas para realizar una mejor discretación de los atributos para realizar la clasificación

In [60]:
dataset.drop(dataset.columns[[1,4,5,6,7,8,11,12]], axis=1, inplace=True)
dataset.head

<bound method NDFrame.head of                 category  dayofweek        date    district      resolution
0           NON-CRIMINAL     Monday  2015-01-19     MISSION            NONE
1                ROBBERY     Sunday  2015-02-01  TENDERLOIN            NONE
2                ASSAULT     Sunday  2015-02-01  TENDERLOIN            NONE
3        SECONDARY CODES     Sunday  2015-02-01  TENDERLOIN            NONE
4              VANDALISM    Tuesday  2015-01-27    NORTHERN            NONE
5           NON-CRIMINAL     Sunday  2015-02-01    RICHMOND            NONE
6        SECONDARY CODES   Saturday  2015-01-31     BAYVIEW            NONE
7              VANDALISM   Saturday  2015-01-31     BAYVIEW            NONE
8               BURGLARY   Saturday  2015-01-31     CENTRAL            NONE
9          LARCENY/THEFT   Saturday  2015-01-31     CENTRAL            NONE
10         LARCENY/THEFT     Sunday  2015-02-01     MISSION  ARREST, BOOKED
11         DRUG/NARCOTIC     Sunday  2015-02-01     MISSIO

* Analizamos las columnas elegidas con el fin de analizar la distancia de la importancia entre ellas

In [61]:
dataset.describe().transpose()

,count,unique,top,freq
category,2215014,39,LARCENY/THEFT,480448
dayofweek,2215014,12,Friday,336117
date,2215014,5618,Wednesday,1989
district,2215013,34,SOUTHERN,398432
resolution,2215014,27,NONE,1378600


## Normalización de datos
La red neuronal en Python puede tener dificultades para converger antes de la cantidad máxima de iteraciones permitidas si los datos no están normalizados. El Perceptron multicapa es sensible a las incrustaciones de características, por lo que es muy recomendable escalar los datos. Hay que tener en cuenta que debe aplicar la misma escala al conjunto de prueba para obtener resultados significativos. Hay muchos métodos diferentes para la normalización de los datos, utilizaremos el StandardScaler incorporado para la estandarización.

In [62]:
ndataset = dataset
categories = { x: ind for ind, x in enumerate(dataset.category.unique()) }
ndataset['category'] = ndataset['category'].map(lambda x: categories[x])

dayofweek = { x: ind for ind, x in enumerate(dataset.dayofweek.unique()) }
ndataset['dayofweek'] = ndataset['dayofweek'].map(lambda x: dayofweek[x])

district = { x: ind for ind, x in enumerate(dataset.district.unique()) }
ndataset['district'] = ndataset['district'].map(lambda x: district[x])

resolution = { x: ind for ind, x in enumerate(dataset.resolution.unique()) }
ndataset['resolution'] = ndataset['resolution'].map(lambda x: resolution[x])

ndataset.head()


,category,dayofweek,date,district,resolution
0,0,0,2015-01-19,0,0
1,1,1,2015-02-01,1,0
2,2,1,2015-02-01,1,0
3,3,1,2015-02-01,1,0
4,4,2,2015-01-27,2,0


Normalización de la columna de fecha

In [63]:
import time
import datetime
def totime(x):
    try:
        return time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple())
    except ValueError:
        return 0

ndataset['date'] = ndataset['date'].map(totime)

ndataset.head()

,category,dayofweek,date,district,resolution
0,0,0,1.421622e+09,0,0
1,1,1,1.422745e+09,1,0
2,2,1,1.422745e+09,1,0
3,3,1,1.422745e+09,1,0
4,4,2,1.422313e+09,2,0


In [64]:
ndataset.drop(ndataset.columns[[2]], axis=1, inplace=True)
ndataset.head

<bound method NDFrame.head of          category  dayofweek  district  resolution
0               0          0         0           0
1               1          1         1           0
2               2          1         1           0
3               3          1         1           0
4               4          2         2           0
5               0          1         3           0
6               3          3         4           0
7               4          3         4           0
8               5          3         5           0
9               6          3         5           0
10              6          1         0           1
11              7          1         0           1
12              7          1         0           1
13              8          1         0           1
14              1          1         0           0
15              9          1         2           0
16              0          1         2           0
17              8          1         4           1
1

### División de muestras y entrenamiento.
Dividamos nuestros datos en conjuntos de entrenamiento y prueba, esto se hace fácilmente con la función train_test_split de SciKit Learn de model_selection:

In [65]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [74]:
x = ndataset.drop('category',axis=1)
y = ndataset['category']
ndataset.head

<bound method NDFrame.head of          category  dayofweek  district  resolution
0               0          0         0           0
1               1          1         1           0
2               2          1         1           0
3               3          1         1           0
4               4          2         2           0
5               0          1         3           0
6               3          3         4           0
7               4          3         4           0
8               5          3         5           0
9               6          3         5           0
10              6          1         0           1
11              7          1         0           1
12              7          1         0           1
13              8          1         0           1
14              1          1         0           0
15              9          1         2           0
16              0          1         2           0
17              8          1         4           1
1

In [67]:
x.head(),y.head()
x_training, x_test, y_training, y_test = train_test_split(x, y)

In [68]:
scaler.fit(x_training)

StandardScaler(copy=True, with_mean=True, with_std=True)

Aplicamos las transformaciones una vez realizada la normalización de los datos

In [73]:
x_training = scaler.transform(x_training)
x_test = scaler.transform(x_test)

x_training
x_test

array([[-2.81277878, -1.96015559, -0.47871692],
       [-2.501272  , -1.9435165 , -0.47400278],
       [-2.62587471, -1.96015559, -0.47400278],
       ..., 
       [-2.87508013, -1.89359924, -0.47871692],
       [-2.81277878, -1.93519696, -0.39386242],
       [-2.501272  , -1.91855787, -0.4645745 ]])

## Entrenador del modelo
Ahora es el momento de entrenar a nuestro modelo. SciKit Learn lo hace increíblemente fácil, mediante el uso de objetos estimadores. En este caso, importaremos nuestro estimador (el modelo clasificador de perceptrón multicapa) de la biblioteca neural_network de SciKit-Learn


In [70]:
from sklearn.neural_network import MLPClassifier

A continuación, creamos una instancia del modelo, hay muchos parámetros que puede elegir definir y personalizar aquí, solo definiremos hidden_layer_sizes. 

* Para este parámetro, se pasa una tupla que consiste en el número de neuronas que se desea en cada capa, donde la enésima entrada de la tupla representa el número de neuronas en la enésima capa del modelo MLP. 
* Hay muchas maneras de elegir estos números, pero para simplificar, elegiremos 3 capas con el mismo número de neuronas que funciones en nuestro conjunto de datos junto con 100 iteraciones máximas, como ejemplo.

In [79]:
classificador = MLPClassifier(hidden_layer_sizes=(10,10,10), max_iter=100,verbose=10)
classificador

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=10, warm_start=False)

In [ ]:
classificador.fit(x_training,y_training)

Iteration 1, loss = 2.66331479
Iteration 2, loss = 2.62095783
Iteration 3, loss = 2.59681230
Iteration 4, loss = 2.59505082
Iteration 5, loss = 2.59477325
Iteration 6, loss = 2.59418773
Iteration 7, loss = 2.59380340
Iteration 8, loss = 2.59343491
Iteration 9, loss = 2.59235191
Iteration 10, loss = 2.59068355


Como podemos ver el resultado muestra los valores predeterminados de los otros parámetros en el modelo. La idea sería ajustar y definir los parámetros de configuración hasta dar con un modelo ajustado a la información a predecir.

## Predicciones y Evaluación
Ahora que tenemos un modelo, es hora de usarlo para obtener predicciones Podemos hacer esto simplemente con el método predict () fuera de nuestro modelo ajustado:

In [ ]:
predictions = classificador.predict(x_test)

Ahora podemos usar las métricas integradas de SciKit-Learn, como un informe de clasificación y una matriz de confusión para evaluar el rendimiento de nuestro modelo:

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))

In [ ]:
print(classification_report(y_test,predictions))

La desventaja de utilizar un modelo Perceptron multicapa es lo difícil que es interpretar el modelo en sí. Los pesos y los sesgos no serán fácilmente interpretables en relación con qué características son importantes para el modelo en sí.

Sin embargo, si desea extraer los pesos y sesgos MLP después de entrenar su modelo, use sus atributos públicos coefs_ e intercepts_.

coefs_ es una lista de matrices de peso, donde la matriz de ponderación en el índice i representa los pesos entre la capa iy la capa i + 1.

intercepts_ es una lista de vectores de sesgo, donde el vector en el índice i representa los valores de sesgo agregados a la capa i + 1.

In [ ]:
len(classificador.coefs_)

In [ ]:
len(classificador.coefs_[0])

In [ ]:
len(classificador.intercepts_[0])

### Conclusión
Intentaremos jugar con la cantidad de capas y neuronas ocultas y observa cómo afectan los resultados de nuestra red neuronal en Python.